In [1]:
import graphlab as gl

In [2]:
song_data = gl.SFrame('song_data.sframe')

This non-commercial license of GraphLab Create for academic use is assigned to smit@uab.edu and will expire on August 06, 2021.


[INFO] graphlab.cython.cy_server: GraphLab Create v2.1 started. Logging: C:\Users\asd\AppData\Local\Temp\graphlab_server_1600108657.log.0


In [3]:
# printing first few values
song_data.head(5)

Columns:
	user_id	str
	song_id	str
	listen_count	int
	title	str
	artist	str
	song	str

Rows: 5

Data:
+-------------------------------+--------------------+--------------+
|            user_id            |      song_id       | listen_count |
+-------------------------------+--------------------+--------------+
| b80344d063b5ccb3212f76538f... | SOAKIMP12A8C130995 |      1       |
| b80344d063b5ccb3212f76538f... | SOBBMDR12A8C13253B |      2       |
| b80344d063b5ccb3212f76538f... | SOBXHDL12A81C204C0 |      1       |
| b80344d063b5ccb3212f76538f... | SOBYHAJ12A6701BF1D |      1       |
| b80344d063b5ccb3212f76538f... | SODACBL12A8C13C273 |      1       |
+-------------------------------+--------------------+--------------+
+-----------------+---------------+-------------------------------+
|      title      |     artist    |              song             |
+-----------------+---------------+-------------------------------+
|     The Cove    |  Jack Johnson |    The Cove - Jack Johnson    |
| Entre Dos Aguas | Paco De Lucia | Entre Dos Aguas - Paco De ... |
|     Stronger    |   Kanye West  |     Stronger - Kanye West     |
|  Constellations |  Jack Johnson | Constellations - Jack Johnson |
|   Learn To Fly  |  Foo Fighters |  Learn To Fly - Foo Fighters  |
+-----------------+---------------+-------------------------------+
[5 rows x 6 columns]

# Count of Unique users in data set

In [47]:
users = song_data['user_id'].unique()
len(users)

66346

# Counting the number of unique users for the artist Kanye West

In [15]:
unique_users_set_1 = song_data[song_data['artist']=='Kanye West']

In [18]:
len(unique_users_set_1['user_id'].unique())

2522

# Counting the number of unique users for the artist Foo Fighters

In [19]:
unique_users_set_2 = song_data[song_data['artist']=='Foo Fighters']

In [20]:
len(unique_users_set_2['user_id'].unique())

2055

# Counting the number of unique users for the artist Taylor Swift

In [21]:
unique_users_set_3 = song_data[song_data['artist']=='Taylor Swift']

In [22]:
len(unique_users_set_3['user_id'].unique())

3246

# Counting the number of unique users for the artist Lady GaGa

In [23]:
unique_users_set_4 = song_data[song_data['artist']=='Lady GaGa']

In [24]:
len(unique_users_set_4['user_id'].unique())

2928

# Using groupby-aggregate to find the most popular and least popular artist

In [35]:
# Most popular artist
listen_song_data = song_data.groupby(key_columns='artist',operations={'total_count':gl.aggregate.SUM('listen_count')}
                                    ).sort('total_count',ascending=False)
listen_song_data

artist,total_count
Kings Of Leon,43218
Dwight Yoakam,40619
Björk,38889
Coldplay,35362
Florence + The Machine,33387
Justin Bieber,29715
Alliance Ethnik,26689
OneRepublic,25754
Train,25402
The Black Keys,22184


In [59]:
# least popular artist
listen_song_data.sort('total_count',ascending=True)

artist,total_count
William Tabbert,14
Reel Feelings,24
Beyoncé feat. Bun B andSlim Thug ...,26
Diplo,30
Boggle Karaoke,30
harvey summers,31
Nâdiya,36
Kanye West / Talib Kweli/ Q-Tip / Common / ...,38
Aneta Langerova,38
Jody Bernal,38


 # Using groupby-aggregate to find the most recommended songs and creating a song recommender

In [36]:
# splitting the data set 20% for test set and 80% for training set
train_data , test_data = song_data.random_split(.8,seed=0)

## Creating a simple personalization based recommender

In [43]:
personalized_base_model = gl.item_similarity_recommender.create(train_data,user_id = 'user_id',item_id = 'song')

Recsys training: model = item_similarity

Warning: Ignoring columns song_id, listen_count, title, artist;

To use one of these as a target column, set target = <column_name>

and use a method that allows the use of a target.

Preparing data set.

...

| 2.29s                               | 100              | 9952            |

+-------------------------------------+------------------+-----------------+

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 2.39144s

## predictions based on item based recommender

In [49]:
# testing first 10000 users only in the test set.
subset_test_users = test_data['user_id'].unique()[0:10000]

In [50]:
personalized_base_model.recommend(subset_test_users,k=1)

recommendations finished on 1000/10000 queries. users per second: 3334.09

recommendations finished on 2000/10000 queries. users per second: 5376.4

recommendations finished on 3000/10000 queries. users per second: 6744.56

recommendations finished on 4000/10000 queries. users per second: 7787.04

recommendations finished on 5000/10000 queries. users per second: 8621.83

...

recommendations finished on 7000/10000 queries. users per second: 9722.36

recommendations finished on 8000/10000 queries. users per second: 10130.3

recommendations finished on 9000/10000 queries. users per second: 10058.1

recommendations finished on 10000/10000 queries. users per second: 9758.13

user_id,song,score,rank
c66c10a9567f0d82ff31441a9fd5063e5cd9dfe8 ...,Cuando Pase El Temblor -Soda Stereo ...,0.0194504536115,1
c067c22072a17d33310d7223d7b79f819e48cf42 ...,Grind With Me (ExplicitVersion) - Pretty Ricky ...,0.0459424376488,1
f6c596a519698c97f1591ad89f540d76f6a04f1a ...,Hey_ Soul Sister - Train,0.0238929539919,1
696787172dd3f5169dc94deef97e427cee86147d ...,Senza Una Donna (WithoutA Woman) - Zucchero / ...,0.017026577677,1
3a7111f4cdf3c5a85fd4053e3cc2333562e1e0cb ...,Heartbreak Warfare - JohnMayer ...,0.0298416515191,1
532e98155cbfd1e1a474a28ed96e59e50f7c5baf ...,Jive Talkin' (AlbumVersion) - Bee Gees ...,0.0118288653237,1
ee43b175ed753b2e2bce806c903d4661ad351a91 ...,Ricordati Di Noi -Valerio Scanu ...,0.0305171211561,1
e372c27f6cb071518ae500589ae02c126954c148 ...,Fall Out - The Police,0.0819672048092,1
83b1428917b47a6b130ed471b09033820be78a8c ...,Clocks - Coldplay,0.042858839035,1
39487deef9345b1e22881245cabf4e7c53b6cf6e ...,Black Mirror - ArcadeFire ...,0.0417737685717,1


## Making recommendations for the users in the test data and we will get the most recommended song to the first 10,000 users in the test data

In [56]:
operations = {'count':gl.aggregate.COUNT()}

number_times_songs_recommended = song_data.groupby(key_columns='song',operations={'total_count':gl.aggregate.SUM('listen_count')}
                                    ).sort('total_count',ascending=False)


In [57]:
number_times_songs_recommended

song,total_count
You're The One - DwightYoakam ...,40619
Undo - Björk,36059
Revelry - Kings Of Leon,30391
Horn Concerto No. 4 in Eflat K495: II. Romance ...,21953
Sehr kosmisch - Harmonia,21646
Dog Days Are Over (RadioEdit) - Florence + The ...,15889
Secrets - OneRepublic,14149
Ain't Misbehavin - SamCooke ...,14029
Invalid - Tub Ring,13701
Catch You Baby (StevePitron & Max Sanna Radio ...,12506
